In [14]:
import numpy as np
import tensorflow as tf
import random
tf.keras.backend.set_floatx('float32')

# Personalized class for O(1) remove and O(1) random access
class ListDict:
    def __init__(self, list):
        self.list = list
        self.dict = {i:i for i in list}
    def remove(self, val):
        index = self.dict[val]
        lastElement = self.list.pop()
        if index != len(self.list):
            self.list[index] = lastElement
            self.dict[lastElement] = index
    def getRandom(self):
        return self.list[random.randint(0, len(self.list) - 1)]

# Self-coded Minesweeper allows for thousands of games to be played in tandem
class MineSweeper:
    def __init__(self, height, width, mines, batchSize):
        self.height = height
        self.width = width
        self.mines = mines
        self.batchSize = batchSize

        # Board is what is inputted into the model and solutionBoard is the y_true. Board is one-hot encoded
        # to distinguish between cells in different states
        self.board = tf.keras.utils.to_categorical(np.full((batchSize, height, width), -1.0), num_classes=10)

        # Most board information is kept flattened as to make calculations easier, 2D representation is only
        # required for convolution layers and visual output. Different numbers represent different values
        # in each board as well in order to simplify calculations
        self.solutionBoard = np.ones((batchSize, height*width))
        self.allowedChoices = np.zeros((batchSize, height*width))
        self.remainingMoves = [ListDict([i for i in range(height*width)]) for _ in range(batchSize)]

        # Optimization strategy for counting the number of mines surrounding a grid is
        # to precompute what the finished board would look like
        # Start out with a padded matrix and use a 3x3 kernel to increment the area surrounding each mine by
        # 1 for each mine and then delete the padding afterwards
        self.solvedBoard = np.zeros((batchSize, height+2, width+2))
        self.kernel = [0, 1, 2, width+2, width+3, width+4, 2*width+4, 2*width+5, 2*width+6]

    def initializeBoard(self, prediction):
        mines = []
        y = prediction // self.width
        x = prediction % self.width
        self.board[:, y, x, 9] = 0
        self.board[:, y, x, 0] = 1
        self.allowedChoices[:, prediction] = 2

        # The first grid that is selected must always be free and not be surrounded by any mines.
        exclude = [i for i in range(max(y*self.width+x-1, y*self.width), min(y*self.width+x+2,(y+1)*self.width))]
        lowerExclude = [i - self.width for i in exclude]
        upperExclude = [i + self.width for i in exclude]
        if y > 0: exclude = lowerExclude + exclude
        if y < self.height-1: exclude = exclude + upperExclude
        choices = np.setdiff1d(range(self.height*self.width), exclude)

        for j in range(self.batchSize):
            self.remainingMoves[j].remove(prediction)
            indices = np.random.choice(choices, replace=False, size=int(self.mines))
            mines.append(indices)
            # Have to account for matrix being padded
            scaledIndices = [i + 2*(i//self.width) for i in indices]
            # Use the index of the mines to move the kernel to the correct spots
            for index in scaledIndices:
                self.solvedBoard[j][np.unravel_index([i + index for i in self.kernel], self.solvedBoard[0].shape)] += 1

        # Unpad matrix
        self.solvedBoard = np.delete(self.solvedBoard, [0,self.height+1], 1)
        self.solvedBoard = np.delete(self.solvedBoard, [0,self.width+1], 2)
        self.solvedBoard = np.reshape(self.solvedBoard, (self.batchSize, self.height*self.width))

        # Set randomized mines
        for j in range(self.batchSize):
            for mine in mines[j]: self.remainingMoves[j].remove(mine)
            self.solvedBoard[j][mines[j]] = 9
            self.solutionBoard[j][mines[j]] = 0
        self.solvedBoard = self.solvedBoard.astype(int)

In [15]:
# Network treats minesweeper as a multilabel multiclassification problem where
# each grid can either be classified as a mine or space, output ideally maintains
# the same shape as the input so we are limited to convolution layers only
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(100,(3,3), activation = 'relu', padding = 'same'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Conv2D(100,(3,3), activation = 'relu', padding = 'same'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Conv2D(100,(3,3), activation = 'relu', padding = 'same'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Conv2D(100,(3,3), activation = 'relu', padding = 'same'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Conv2D(100,(3,3), activation = 'relu', padding = 'same'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Conv2D(1,(3,3), activation = 'sigmoid', padding = 'same'),
])
optimizer = tf.keras.optimizers.RMSprop()

In [16]:
# Model accuracy during training performs better than testing as it is learning off
# of the given solution sets, therefore we compute validation accuracy seperately
def getAccuracy(model, height, width, mines, batchSize):
    wins = np.repeat(True, batchSize)
    sweep = MineSweeper(height, width, mines, batchSize)
    y_pred = tf.reshape(model(sweep.board), (batchSize, height*width))
    sweep.initializeBoard(tf.math.argmin(y_pred, axis=1).numpy()[0])
    numMoves = height*width - mines - 1

    for i in range(numMoves):
        y_pred = tf.reshape(model(sweep.board), (batchSize, height*width))
        choices = tf.math.argmax(y_pred - sweep.allowedChoices, axis=1).numpy()
        for j, choice in enumerate(choices):
            if sweep.solutionBoard[j][choice] == 0:
                choices[j] = sweep.remainingMoves[j].getRandom()
                wins[j] = False
        for j, choice in enumerate(choices):
            sweep.remainingMoves[j].remove(choice)
            sweep.allowedChoices[j][choice] = 2
            sweep.board[j][choice//width][choice%width][9] = 0
            sweep.board[j][choice//width][choice%width][sweep.solvedBoard[j][choice]] = 1
    return wins.sum()/(batchSize)

# Customized gradient descent loop is required in order to update the Minesweeper boards during training
# Model training is done by playing hundreds to thousands of games in tandem in accordance to the batch size,
# every gradient descent loop it will predict a single move for each board and then run gradient descent
# over that single move. This will be done until each game is complete then it will move onto the next batch of games.
def trainModel(height, width, mines, games, batchSize, modelName):
    numMoves = height*width - mines - 1
    accuracy = []
    wins = np.full((games, batchSize), True)

    for game in range(games):
        loss_cum = 0
        sweep = MineSweeper(height, width, mines, batchSize)

        # In classic minesweeper, the board is only initialized once you click on a space
        # Therefore, the first prediction must be made before initializing the board
        with tf.GradientTape() as tape:
            y_pred = tf.reshape(model(sweep.board), (batchSize, height*width))
            sweep.initializeBoard(tf.math.argmin(y_pred, axis=1).numpy()[0])
            loss = tf.keras.losses.BinaryCrossentropy()(sweep.solutionBoard, y_pred)
        grads = tape.gradient(loss, model.trainable_weights)
        optimizer.apply_gradients(zip(grads, model.trainable_weights))

        # Due to the nature of the model it will perform the same move more than once
        # A simple preprocessing step using the allowedChoices board prevents this from happening
        for i in range(numMoves):
            with tf.GradientTape() as tape:
                y_pred = tf.reshape(model(sweep.board), (batchSize, height*width))
                choices = tf.math.argmax(y_pred - sweep.allowedChoices, axis=1).numpy()
                for j, choice in enumerate(choices):
                    if sweep.solutionBoard[j][choice] == 0:
                        choices[j] = sweep.remainingMoves[j].getRandom()
                        wins[game][j] = False
                for j, choice in enumerate(choices):
                    sweep.remainingMoves[j].remove(choice)
                    sweep.allowedChoices[j][choice] = 2
                    sweep.board[j][choice//width][choice%width][9] = 0
                    sweep.board[j][choice//width][choice%width][sweep.solvedBoard[j][choice]] = 1
                loss = tf.keras.losses.BinaryCrossentropy()(sweep.solutionBoard, y_pred)
                loss_cum += loss
            grads = tape.gradient(loss, model.trainable_weights)
            optimizer.apply_gradients(zip(grads, model.trainable_weights))

        valWinRate = round(100 * getAccuracy(model, height, width, mines, batchSize), 2)
        winRate = round(100 * wins[game].sum()/batchSize, 2)
        print("Batch " + str(game + 1) + " loss = " + str(round(loss_cum.numpy(), 2))
                + "   Win rate: " + str(winRate) + "%" + "   Val Win rate: " + str(valWinRate) + "%")
        if game != 0 and valWinRate > np.max(accuracy):
            model.save(modelName + '.keras')
        accuracy.append(valWinRate)
    return accuracy

In [17]:
# Beginner, intermediate, and expert level boards preset and pretrained
height, width, mines, games, batchSize = (9, 9, 10, 100, 2048)
beginnerAccuracy = trainModel(height, width, mines, games, batchSize, modelName='BeginnerModel')

Batch 1 loss = 23.35   Win rate: 0.0%   Val Win rate: 0.05%
Batch 2 loss = 15.88   Win rate: 15.97%   Val Win rate: 16.5%
Batch 3 loss = 14.39   Win rate: 39.26%   Val Win rate: 44.09%
Batch 4 loss = 13.11   Win rate: 62.74%   Val Win rate: 63.38%
Batch 5 loss = 12.56   Win rate: 73.19%   Val Win rate: 74.02%
Batch 6 loss = 12.29   Win rate: 80.08%   Val Win rate: 77.73%
Batch 7 loss = 12.06   Win rate: 82.71%   Val Win rate: 80.91%
Batch 8 loss = 11.98   Win rate: 85.11%   Val Win rate: 82.32%
Batch 9 loss = 11.86   Win rate: 86.08%   Val Win rate: 84.28%
Batch 10 loss = 11.85   Win rate: 88.43%   Val Win rate: 86.08%
Batch 11 loss = 11.78   Win rate: 87.01%   Val Win rate: 86.23%
Batch 12 loss = 11.68   Win rate: 88.62%   Val Win rate: 83.35%
Batch 13 loss = 12.02   Win rate: 87.26%   Val Win rate: 87.26%
Batch 14 loss = 11.66   Win rate: 90.23%   Val Win rate: 88.18%
Batch 15 loss = 11.6   Win rate: 91.55%   Val Win rate: 89.01%
Batch 16 loss = 11.82   Win rate: 91.46%   Val Win rat

In [28]:
height, width, mines, games, batchSize = (16, 16, 40, 20, 2048)
intermediateAccuracy = trainModel(height, width, mines, games, batchSize, modelName='IntermediateModel')

Batch 1 loss = 43.76   Win rate: 81.4%   Val Win rate: 77.25%
Batch 2 loss = 43.57   Win rate: 84.18%   Val Win rate: 78.71%
Batch 3 loss = 43.22   Win rate: 84.81%   Val Win rate: 78.12%
Batch 4 loss = 43.49   Win rate: 85.69%   Val Win rate: 80.81%
Batch 5 loss = 43.26   Win rate: 86.13%   Val Win rate: 80.57%
Batch 6 loss = 42.97   Win rate: 86.04%   Val Win rate: 78.66%
Batch 7 loss = 43.29   Win rate: 83.69%   Val Win rate: 77.98%
Batch 8 loss = 43.0   Win rate: 87.7%   Val Win rate: 80.32%
Batch 9 loss = 43.15   Win rate: 86.82%   Val Win rate: 77.54%
Batch 10 loss = 43.14   Win rate: 84.42%   Val Win rate: 81.84%
Batch 11 loss = 43.58   Win rate: 85.89%   Val Win rate: 80.22%
Batch 12 loss = 43.18   Win rate: 84.38%   Val Win rate: 77.1%
Batch 13 loss = 42.88   Win rate: 82.23%   Val Win rate: 81.64%
Batch 14 loss = 42.97   Win rate: 86.38%   Val Win rate: 81.1%
Batch 15 loss = 43.49   Win rate: 84.96%   Val Win rate: 81.35%
Batch 16 loss = 43.18   Win rate: 87.6%   Val Win rate

In [30]:
height, width, mines, games, batchSize = (16, 30, 99, 15, 2048)
expertAccuracy = trainModel(height, width, mines, games, batchSize, modelName='ExpertModel')

Batch 1 loss = 94.59   Win rate: 41.94%   Val Win rate: 27.1%
Batch 2 loss = 94.53   Win rate: 45.31%   Val Win rate: 30.71%
Batch 3 loss = 94.54   Win rate: 45.36%   Val Win rate: 30.42%
Batch 4 loss = 94.66   Win rate: 48.19%   Val Win rate: 29.79%
Batch 5 loss = 94.26   Win rate: 42.87%   Val Win rate: 30.71%
Batch 6 loss = 94.29   Win rate: 43.85%   Val Win rate: 30.96%
Batch 7 loss = 94.22   Win rate: 42.48%   Val Win rate: 30.22%
Batch 8 loss = 94.16   Win rate: 46.04%   Val Win rate: 30.71%
Batch 9 loss = 94.3   Win rate: 43.12%   Val Win rate: 33.3%
Batch 10 loss = 94.29   Win rate: 50.29%   Val Win rate: 32.23%
Batch 11 loss = 94.13   Win rate: 46.78%   Val Win rate: 31.93%
Batch 12 loss = 93.98   Win rate: 46.29%   Val Win rate: 31.25%
Batch 13 loss = 94.55   Win rate: 46.14%   Val Win rate: 30.96%
Batch 14 loss = 93.96   Win rate: 48.54%   Val Win rate: 33.01%
Batch 15 loss = 94.02   Win rate: 47.07%   Val Win rate: 33.11%


In [33]:
beginnerModel = tf.keras.models.load_model('BeginnerModel.keras')
print("Beginner board: " + str(100 * getAccuracy(beginnerModel, 9, 9, 10, 2048)) + "% win rate")
intermediateModel = tf.keras.models.load_model('IntermediateModel.keras')
print("Intermediate board: " + str(100 * getAccuracy(intermediateModel, 16, 16, 40, 2048)) + "% win rate")
expertModel = tf.keras.models.load_model('ExpertModel.keras')
print("Expert board: " + str(100 * getAccuracy(expertModel, 16, 30, 99, 2048)) + "% win rate")

Beginner board: 95.01953125% win rate
Intermediate board: 81.54296875% win rate
Expert board: 33.203125% win rate
